## Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
forest = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Forest_Data/full_forest')
forest.head()

,Unnamed: 0,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_01,Wilderness_Area_02,Wilderness_Area_03,Wilderness_Area_04,Soil_Type_01,Soil_Type_02,Soil_Type_03,Soil_Type_04,Soil_Type_05,Soil_Type_06,Soil_Type_07,Soil_Type_08,Soil_Type_09,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type,Wilderness,Soil
0,0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29
1,1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29
2,2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,12
3,3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,30
4,4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29


Removing index column and dummy columns of soil_types and wilderness_areas and storing it in a dataframe **'forest_a'** 

In [ ]:
forest = forest.drop(columns = forest.columns[:1])
forest_a = forest.drop(columns = forest.columns[10:54])

Combining the cover_type: 3,4,6 as '3' and cover_type: 5,7 as '4'.

In [ ]:
forest_a['Cover_Type'] = np.where(((forest_a['Cover_Type']==4)|(forest_a['Cover_Type']==6)),3,forest_a['Cover_Type'])
forest_a['Cover_Type'] = np.where(((forest_a['Cover_Type']==5)|(forest_a['Cover_Type']==7)),4,forest_a['Cover_Type'])

Converting soil, wilderness and cover_type columns to category

In [ ]:
for i in forest.columns[::-1][:3]:
  forest_a[i] = forest_a[i].astype('category')

In [ ]:
forest_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  int64   
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64   
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64   
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64   
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64   
 10  Cover_Type                          581012 non-null  category
 11  Wilderness   

## Modelling on Original Data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_a.drop(columns = ['Cover_Type'])
y = forest_a['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9360966590280988
Testing Score: 0.9114248668992105


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.91      0.92      0.92     85045
           3       0.96      0.96      0.96     16786
           4       0.90      0.86      0.88      8959

    accuracy                           0.91    174304
   macro avg       0.92      0.91      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56899  6184    21   410]
 [ 5904 78199   503   439]
 [   17   649 16093    27]
 [  523   690    72  7674]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.431120
1,Aspect,0.017520
2,Slope,0.009448
3,Horizontal_Distance_To_Hydrology,0.047408
4,Vertical_Distance_To_Hydrology,0.030738
5,Horizontal_Distance_To_Roadways,0.123607
6,Hillshade_9am,0.021908
7,Hillshade_Noon,0.023617
8,Hillshade_3pm,0.014067
9,Horizontal_Distance_To_Fire_Points,0.120515


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train,y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9532342614357229
Testing Score: 0.9323136588948044


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.90      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.97      0.97      0.97     16786
           4       0.96      0.82      0.89      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.91      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[57451  5867    16   180]
 [ 3104 81349   464   128]
 [    1   435 16346     4]
 [  584   948    67  7360]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.304111
1,Aspect,0.027166
2,Slope,0.019277
3,Horizontal_Distance_To_Hydrology,0.041677
4,Vertical_Distance_To_Hydrology,0.036109
5,Horizontal_Distance_To_Roadways,0.107407
6,Hillshade_9am,0.023126
7,Hillshade_Noon,0.024934
8,Hillshade_3pm,0.021661
9,Horizontal_Distance_To_Fire_Points,0.098111


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_a.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_a['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_a[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,29,1
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,29,1
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,12,1
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,30,1
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9449639709932073


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.95      0.95     85045
           3       0.96      0.97      0.97     16786
           4       0.92      0.90      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.94      0.94      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[59348  3862    19   285]
 [ 3213 80952   521   359]
 [   16   374 16353    43]
 [  353   489    59  8058]]


### Ensemble Techniques on Original Data

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9422632453750602
Training Score: 0.9656166094593664
Testing Score: 0.9448664402423352


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.95      0.87      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[58938  4353     5   218]
 [ 2909 81596   348   192]
 [   12   359 16407     8]
 [  458   672    76  7753]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9194335985522782
Training Score: 0.9370457428917061
Accuracy: 0.9200649440058748


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     63514
           2       0.90      0.95      0.93     85045
           3       0.96      0.97      0.96     16786
           4       0.96      0.78      0.86      8959

    accuracy                           0.92    174304
   macro avg       0.94      0.90      0.91    174304
weighted avg       0.92      0.92      0.92    174304

[[56542  6760    17   195]
 [ 3711 80626   585   123]
 [    1   534 16249     2]
 [  730  1206    69  6954]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9438171857942308
Training Score: 0.9998426389448941
Testing Score: 0.9445050027538094


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.95      0.95     85045
           3       0.96      0.97      0.97     16786
           4       0.92      0.90      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.94      0.94      0.94    174304
weighted avg       0.94      0.94      0.94    174304



### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                    

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 1.0
AdaBoost DT Testing Score: 0.9591518266935928


#### Random Forest

In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.95     63514
           2       0.95      0.97      0.96     85045
           3       0.98      0.98      0.98     16786
           4       0.97      0.89      0.93      8959

    accuracy                           0.96    174304
   macro avg       0.97      0.95      0.96    174304
weighted avg       0.96      0.96      0.96    174304

[[60179  3227     4   104]
 [ 2131 82514   311    89]
 [    4   261 16508    13]
 [  340   582    54  7983]]


In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9226324537506024
AdaBoost RF Testing Score: 0.8851259867817147


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.87      0.85      0.86     63514
           2       0.88      0.92      0.90     85045
           3       0.96      0.97      0.97     16786
           4       0.97      0.69      0.81      8959

    accuracy                           0.89    174304
   macro avg       0.92      0.86      0.88    174304
weighted avg       0.89      0.89      0.88    174304

[[53703  9706     4   101]
 [ 6338 78013   607    87]
 [    2   410 16362    12]
 [ 1740   969    47  6203]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9734601615568202


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.92      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.97    174304
weighted avg       0.97      0.97      0.97    174304

[[61517  1869     2   126]
 [ 1364 83342   205   134]
 [    2   221 16548    15]
 [  255   383    50  8271]]


### Stacking

In [ ]:
from mlxtend.classifier import StackingClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
sclf = StackingClassifier(classifiers=[knn, rforest, abcl_DT], 
                          meta_classifier=gbcl)
sclf.fit(x_train,y_train)

StackingClassifier(average_probas=False,
                   classifiers=[KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='cityblock',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=4,
                                                     p=2, weights='distance'),
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='entropy',
                                                       max_depth=19,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
   

In [ ]:
print('Training Score:',sclf.score(x_train,y_train))
print('Testing Score:',sclf.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9602131907472002


In [ ]:
print(classification_report(y_test,sclf.predict(x_test)))
print(confusion_matrix(y_test,sclf.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.95     63514
           2       0.95      0.97      0.96     85045
           3       0.99      0.99      0.99     16786
           4       0.98      0.88      0.93      8959

    accuracy                           0.96    174304
   macro avg       0.97      0.95      0.96    174304
weighted avg       0.96      0.96      0.96    174304

[[60221  3225     5    63]
 [ 2133 82676   180    56]
 [    3   190 16580    13]
 [  388   624    55  7892]]


## Feature Engineering

Min scaling 'vertical_distance_to_hydrology' column as it has got some negative values

In [ ]:
forest_a['Vertical_Scaled'] = forest_a['Vertical_Distance_To_Hydrology'] - min(forest_a['Vertical_Distance_To_Hydrology'])

Calculating Euclidean distance to hydrology. As water being the necessity for a vegetation to grow.

In [ ]:
forest_a['Hydr_Dist'] = (forest_a['Horizontal_Distance_To_Hydrology']**2 + forest_a['Vertical_Distance_To_Hydrology']**2)**0.5

Taking average of all the Hillshades

In [ ]:
forest_a['Hillshades'] = (forest_a['Hillshade_3pm'] + forest_a['Hillshade_Noon'] + forest_a['Hillshade_9am'])/3

Taking Linear Combination of 'Horizontal_distance_to_roadways and 'Horizontal_distance_to_Firepoints'.

In [ ]:
forest_a['Fire_Road'] = forest_a['Horizontal_Distance_To_Fire_Points'] + forest_a['Horizontal_Distance_To_Roadways']

Taking average of all the horizontal distances columns. Fire_points, hydrology and roadways being the basic amenities affecting any vegetation.

In [ ]:
forest_a['Avg_Horizontal'] = (forest_a['Horizontal_Distance_To_Fire_Points'] + 
                              forest_a['Horizontal_Distance_To_Hydrology'] + 
                              forest_a['Horizontal_Distance_To_Roadways'])/3

Taking the average of 'Horizotal_Distance_to_Fire_points' and 'Horizontal_distance_to_hydrology' column.</b>

In [ ]:
forest_a['Fire_Hydr'] = (forest_a['Horizontal_Distance_To_Fire_Points'] + forest_a['Horizontal_Distance_To_Hydrology'])/2

Calculating at what steepness the particular cover_type is located from hydrology

In [ ]:
forest_a['Hydr_Slope'] = forest_a['Vertical_Distance_To_Hydrology']/forest_a['Horizontal_Distance_To_Hydrology']

converting the slope values into degrees and replacing all the null values with 0

In [ ]:
from math import degrees
a = []
for i in forest_a['Hydr_Slope']:
  a.append(degrees(np.arctan(i)))

b = pd.DataFrame({'a':a})
forest_a['Hydr_Slope'] = np.where(b['a'].isnull(),0,b['a'])
forest_a['Hydr_Slope'].isnull().sum()  

Making a list of all numerical Columns and storing it in a variable 'quant_col'

In [ ]:
quant_col = list(forest_a.columns[:10])
quant_col.extend(forest_a.columns[::-1][:7])
quant_col

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Hydr_Slope',
 'Fire_Hydr',
 'Avg_Horizontal',
 'Fire_Road',
 'Hillshades',
 'Hydr_Dist',
 'Vertical_Scaled']

In [ ]:
forest_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  int64   
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64   
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64   
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64   
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64   
 10  Cover_Type                          581012 non-null  category
 11  Wilderness   

## Modelling after Feature Engineering

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_a.drop(columns = ['Cover_Type','Vertical_Scaled'])
y = forest_a['Cover_Type'].astype('category')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9402421393235442
Testing Score: 0.9152343032862126


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.91      0.90      0.90     63514
           2       0.91      0.93      0.92     85045
           3       0.96      0.96      0.96     16786
           4       0.91      0.85      0.88      8959

    accuracy                           0.92    174304
   macro avg       0.92      0.91      0.92    174304
weighted avg       0.92      0.92      0.92    174304

[[57010  6190     8   306]
 [ 5360 78713   566   406]
 [   14   579 16163    30]
 [  573   662    81  7643]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.423821
1,Aspect,0.013237
2,Slope,0.006039
3,Horizontal_Distance_To_Hydrology,0.013497
4,Vertical_Distance_To_Hydrology,0.018565
5,Horizontal_Distance_To_Roadways,0.081928
6,Hillshade_9am,0.020043
7,Hillshade_Noon,0.017263
8,Hillshade_3pm,0.009920
9,Horizontal_Distance_To_Fire_Points,0.052746


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train,y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9661255741219745
Testing Score: 0.9468055810537911


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.96      0.86      0.91      8959

    accuracy                           0.95    174304
   macro avg       0.96      0.93      0.94    174304
weighted avg       0.95      0.95      0.95    174304

[[58812  4550     8   144]
 [ 2483 82034   387   141]
 [    0   309 16473     4]
 [  437   737    72  7713]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.281791
1,Aspect,0.019296
2,Slope,0.011800
3,Horizontal_Distance_To_Hydrology,0.021301
4,Vertical_Distance_To_Hydrology,0.020570
5,Horizontal_Distance_To_Roadways,0.066244
6,Hillshade_9am,0.018238
7,Hillshade_Noon,0.016131
8,Hillshade_3pm,0.015084
9,Horizontal_Distance_To_Fire_Points,0.048492


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_a.drop(columns= ['Cover_Type','Soil','Wilderness','Vertical_Scaled'])
y_ = forest_a['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_a[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,-0.083233,0.530152,1.043224,1.027143,3.171097,-1.041587,29,1
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,-0.294777,0.645367,0.969391,0.934802,3.097132,-1.228740,29,1
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,-0.001362,0.668410,2.109122,2.085710,3.061629,0.532279,12,1
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,-0.031466,0.461022,2.109122,2.074797,3.108966,1.959275,30,1
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,-0.566983,0.599281,0.947327,0.888212,3.014291,-1.084819,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9537589498806682


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.95      0.95      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.97      0.98      0.97     16786
           4       0.93      0.92      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.95      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[60038  3189    22   265]
 [ 2683 81605   428   329]
 [   15   335 16392    44]
 [  306   391    53  8209]]


### Ensemble Techniques

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.946189895453249
Training Score: 0.9693662283505611
Testing Score: 0.9495020194602534


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.95      0.88      0.91      8959

    accuracy                           0.95    174304
   macro avg       0.96      0.94      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59363  3967     6   178]
 [ 2696 81792   358   199]
 [    7   293 16481     5]
 [  413   611    69  7866]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9346312342024253
Training Score: 0.951230366749609
Accuracy: 0.9348838810354323


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.93     63514
           2       0.92      0.96      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.82      0.88      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.91      0.93    174304
weighted avg       0.94      0.93      0.93    174304

[[57850  5488     9   167]
 [ 3017 81421   477   130]
 [    1   403 16380     2]
 [  607   975    74  7303]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9515918054230553
Training Score: 0.9998844379751566
Testing Score: 0.952967229667707


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.97      0.98      0.97     16786
           4       0.93      0.91      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.95      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59948  3292    19   255]
 [ 2657 81609   459   320]
 [   14   354 16381    37]
 [  323   408    60  8168]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 1.0
AdaBoost DT Testing Score: 0.9615097760234992


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.96     63514
           2       0.96      0.97      0.96     85045
           3       0.98      0.99      0.98     16786
           4       0.98      0.90      0.94      8959

    accuracy                           0.96    174304
   macro avg       0.97      0.95      0.96    174304
weighted avg       0.96      0.96      0.96    174304

[[60354  3073     0    87]
 [ 2026 82648   282    89]
 [    1   231 16543    11]
 [  317   545    47  8050]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9542398969285089
AdaBoost RF Testing Score: 0.9191240591151092


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.91      0.90      0.90     63514
           2       0.91      0.94      0.93     85045
           3       0.97      0.98      0.98     16786
           4       0.98      0.79      0.87      8959

    accuracy                           0.92    174304
   macro avg       0.94      0.90      0.92    174304
weighted avg       0.92      0.92      0.92    174304

[[56960  6485     3    66]
 [ 4835 79675   446    89]
 [    2   279 16495    10]
 [ 1117   718    47  7077]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9759730126675235


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.93      0.95      8959

    accuracy                           0.98    174304
   macro avg       0.98      0.97      0.97    174304
weighted avg       0.98      0.98      0.98    174304

[[61694  1707     2   111]
 [ 1207 83504   209   125]
 [    1   188 16587    10]
 [  218   368    42  8331]]


## Added Features

Constructing a dataframe with added features and removing features on which some feature engineering has been applied, so as to gauge their performance in classifying cover_types.

In [ ]:
forest_z = forest_a.drop(columns = forest_a.columns[3:10])
forest_z = forest_z.drop(columns = ['Vertical_Scaled'])
forest_z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Elevation       581012 non-null  int64   
 1   Aspect          581012 non-null  int64   
 2   Slope           581012 non-null  int64   
 3   Cover_Type      581012 non-null  category
 4   Wilderness      581012 non-null  category
 5   Soil            581012 non-null  category
 6   Hydr_Dist       581012 non-null  float64 
 7   Hillshades      581012 non-null  float64 
 8   Fire_Road       581012 non-null  int64   
 9   Avg_Horizontal  581012 non-null  float64 
 10  Fire_Hydr       581012 non-null  float64 
 11  Hydr_Slope      581012 non-null  float64 
dtypes: category(3), float64(5), int64(4)
memory usage: 41.6 MB


## Modelling on added features

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_z.drop(columns = ['Cover_Type'])
y = forest_z['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9327625716730431
Testing Score: 0.9060492013952635


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.89      0.89      0.89     63514
           2       0.91      0.91      0.91     85045
           3       0.95      0.96      0.95     16786
           4       0.89      0.85      0.87      8959

    accuracy                           0.91    174304
   macro avg       0.91      0.90      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56617  6486     9   402]
 [ 6087 77625   788   545]
 [   21   652 16087    26]
 [  555   721    84  7599]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.433052
1,Aspect,0.038102
2,Slope,0.012361
3,Wilderness,0.045450
4,Soil,0.119650
5,Hydr_Dist,0.048117
6,Hillshades,0.025277
7,Fire_Road,0.087597
8,Avg_Horizontal,0.059384
9,Fire_Hydr,0.098327


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9567798027085772
Testing Score: 0.936662382963099


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.93     63514
           2       0.93      0.96      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.84      0.90      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.92      0.93    174304
weighted avg       0.94      0.94      0.94    174304

[[57974  5365     7   168]
 [ 3128 81334   449   134]
 [    0   356 16424     6]
 [  510   841    76  7532]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.297067
1,Aspect,0.037686
2,Slope,0.017801
3,Wilderness,0.096586
4,Soil,0.194250
5,Hydr_Dist,0.046365
6,Hillshades,0.027096
7,Fire_Road,0.081096
8,Avg_Horizontal,0.082020
9,Fire_Hydr,0.085018


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_z.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_z['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_z[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.083233,0.530152,1.043224,1.027143,3.171097,-1.041587,29,1
1,-1.319235,-0.890480,-1.616363,-0.294777,0.645367,0.969391,0.934802,3.097132,-1.228740,29,1
2,-0.554907,-0.148836,-0.681563,-0.001362,0.668410,2.109122,2.085710,3.061629,0.532279,12,1
3,-0.622768,-0.005869,0.520322,-0.031466,0.461022,2.109122,2.074797,3.108966,1.959275,30,1
4,-1.301377,-0.988770,-1.616363,-0.566983,0.599281,0.947327,0.888212,3.014291,-1.084819,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.954447402239765


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.95      0.95      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.97      0.98      0.98     16786
           4       0.93      0.92      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.95      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[60029  3201    19   265]
 [ 2622 81705   403   315]
 [   14   309 16428    35]
 [  315   392    50  8202]]


### Ensemble techniques

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9415231566627654
Training Score: 0.9656780786215171
Testing Score: 0.9440116118964568


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.97     16786
           4       0.95      0.87      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[58956  4352     6   200]
 [ 3083 81337   436   189]
 [    8   339 16429    10]
 [  417   646    73  7823]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9262689693834397
Training Score: 0.9430057928538411
Accuracy: 0.9266167156232789


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.90      0.92     63514
           2       0.91      0.95      0.93     85045
           3       0.96      0.97      0.97     16786
           4       0.95      0.80      0.87      8959

    accuracy                           0.93    174304
   macro avg       0.94      0.91      0.92    174304
weighted avg       0.93      0.93      0.93    174304

[[57205  6090    10   209]
 [ 3520 80816   580   129]
 [    0   452 16331     3]
 [  646  1080    72  7161]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9528506938639023
Training Score: 0.9998942730411008
Testing Score: 0.954338397282908


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.97      0.98      0.97     16786
           4       0.93      0.91      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.95      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59969  3264    15   266]
 [ 2546 81767   417   315]
 [   12   301 16438    35]
 [  324   400    64  8171]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 1.0
AdaBoost DT Testing Score: 0.958279787038737


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.95     63514
           2       0.95      0.97      0.96     85045
           3       0.98      0.98      0.98     16786
           4       0.98      0.89      0.93      8959

    accuracy                           0.96    174304
   macro avg       0.97      0.95      0.96    174304
weighted avg       0.96      0.96      0.96    174304

[[60167  3252     1    94]
 [ 2239 82370   339    97]
 [    3   259 16514    10]
 [  362   569    47  7981]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.931744642347827
AdaBoost RF Testing Score: 0.9010636588948044


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.90      0.86      0.88     63514
           2       0.89      0.93      0.91     85045
           3       0.96      0.98      0.97     16786
           4       0.97      0.74      0.84      8959

    accuracy                           0.90    174304
   macro avg       0.93      0.88      0.90    174304
weighted avg       0.90      0.90      0.90    174304

[[54478  8944     3    89]
 [ 4825 79501   616   103]
 [    1   298 16475    12]
 [ 1402   908    44  6605]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9723701119882504


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.97     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.93      0.95      8959

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.97    174304
weighted avg       0.97      0.97      0.97    174304

[[61384  1992     5   133]
 [ 1435 83235   235   140]
 [    4   193 16581     8]
 [  260   372    39  8288]]


## Transformations nd Outliers



*   Making three new dataframes forest_sqrt, forest_log, forest_cos
*   Storing the square root transformation of all numerical columns in forest_sqrt
*   Storing the log transformation of all numerical columns in forest_log
*   Storing the cos transformation of all numerical columns in forest_cos



In [ ]:
forest_sqrt = pd.DataFrame()
forest_log = pd.DataFrame()
forest_cos = pd.DataFrame()
for i in quant_col:
  forest_sqrt[i] = np.sqrt(forest_a[i])
  forest_log[i] = np.log(forest_a[i]+1)
  forest_cos[i] = np.cos(forest_a[i])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Calculating the number of outliers in each numerical columns of original data after feature engineering

In [ ]:
outliers = []
for i in quant_col:
    q1 = forest_a[i].quantile(0.25)
    q3 = forest_a[i].quantile(0.75)
    IQR = q3- q1
    outlier = forest_a[i][(forest_a[i] < (q1 - IQR*1.5)) | (forest_a[i] > (q3 + 1.5*IQR))].count()
    outliers.append(outlier)

Calculating the number of outliers in each numerical columns of square root transformed data

In [ ]:
outliers_sqrt = []
for i in quant_col:
    q1 = forest_sqrt[i].quantile(0.25)
    q3 = forest_sqrt[i].quantile(0.75)
    IQR = q3- q1
    outlier_sqrt = forest_sqrt[i][(forest_sqrt[i] < (q1 - IQR*1.5)) | (forest_sqrt[i] > (q3 + 1.5*IQR))].count()
    outliers_sqrt.append(outlier_sqrt)

Calculating the number of outliers in each numerical columns of log transformed data

In [ ]:
outliers_log = []
for i in quant_col:
    q1 = forest_log[i].quantile(0.25)
    q3 = forest_log[i].quantile(0.75)
    IQR = q3- q1
    outlier_log = forest_log[i][(forest_log[i] < (q1 - IQR*1.5)) | (forest_log[i] > (q3 + 1.5*IQR))].count()
    outliers_log.append(outlier_log)

Calculating the number of outliers in each numerical columns of cos transformed data

In [ ]:
outliers_cos = []
for i in quant_col:
    q1 = forest_cos[i].quantile(0.25)
    q3 = forest_cos[i].quantile(0.75)
    IQR = q3- q1
    outlier_cos = forest_cos[i][(forest_cos[i] < (q1 - IQR*1.5)) | (forest_cos[i] > (q3 + 1.5*IQR))].count()
    outliers_cos.append(outlier_cos)

Constructing a new dataframe and listing the number of outliers after each transformation to their respective columns

In [ ]:
outliers = pd.DataFrame({'columns': quant_col, 'outlier':outliers, 
                          'sqrt_outlier': outliers_sqrt, 
                          'log_outlier': outliers_log,
                          'cos_outlier':outliers_cos})
outliers

,columns,outlier,sqrt_outlier,log_outlier,cos_outlier
0,Elevation,15569,19308,22889,0
1,Aspect,0,0,14762,0
2,Slope,15316,7331,12186,0
3,Horizontal_Distance_To_Hydrology,14557,1262,24603,0
4,Vertical_Distance_To_Hydrology,31463,4380,4858,0
5,Horizontal_Distance_To_Roadways,669,0,10940,0
6,Hillshade_9am,17433,21984,26073,0
7,Hillshade_Noon,15672,18866,21368,0
8,Hillshade_3pm,7832,14563,21476,0
9,Horizontal_Distance_To_Fire_Points,31157,8120,11907,0


*   Will be taking Elevation, Aspect and all Hillshade columns original whereas the degree columns (slope & hydr_slope) to be cos transformed, vertical_scaled log transformed and others square root transformed; so as to have least outliers in each column.
*   We won't be dropping any outliers because they contain major data of least distributed cover_types.



## Transformed Data

In [ ]:
forest_b = pd.DataFrame()
forest_b['Elevation'] = forest_a['Elevation']
forest_b['Aspect'] = forest_a['Aspect']
forest_b['Slope'] = forest_cos['Slope']
forest_b['Horizontal_Distance_To_Hydrology'] = forest_sqrt['Horizontal_Distance_To_Hydrology']
forest_b['Vertical_Scaled'] = forest_log['Vertical_Scaled']
forest_b['Horizontal_Distance_To_Roadways'] = forest_sqrt['Horizontal_Distance_To_Roadways']
forest_b['Hillshade_9am'] = forest_a['Hillshade_9am']
forest_b['Hillshade_Noon'] = forest_a['Hillshade_Noon']
forest_b['Hillshade_3pm'] = forest_a['Hillshade_3pm']
forest_b['Horizontal_Distance_To_Fire_Points'] = forest_sqrt['Horizontal_Distance_To_Fire_Points']
forest_b['Hydr_Slope'] = forest_cos['Hydr_Slope']
forest_b['Avg_Horizontal'] = forest_sqrt['Avg_Horizontal']
forest_b['Fire_Road'] = forest_sqrt['Fire_Road']
forest_b['Hillshades'] = forest_a['Hillshades']
forest_b['Hydr_Dist'] = forest_sqrt['Hydr_Dist']
forest_b['Fire_Hydr'] = forest_sqrt['Fire_Hydr']
forest_b['Wilderness'] = forest_a['Wilderness'].astype('category')
forest_b['Soil'] = forest_a['Soil'].astype('category')
forest_b['Cover_Type'] = forest_a['Cover_Type'].astype('category')

In [ ]:
forest_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    581012 non-null  float64 
 4   Vertical_Scaled                     581012 non-null  float64 
 5   Horizontal_Distance_To_Roadways     581012 non-null  float64 
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  float64 
 10  Hydr_Slope                          581012 non-null  float64 
 11  Avg_Horizonta

## Modelling after Transforming data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_b.drop(columns = ['Cover_Type'])
y = forest_b['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9404044179116221
Testing Score: 0.9161465026620158


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.91      0.90      0.90     63514
           2       0.92      0.93      0.92     85045
           3       0.96      0.96      0.96     16786
           4       0.91      0.86      0.88      8959

    accuracy                           0.92    174304
   macro avg       0.92      0.91      0.92    174304
weighted avg       0.92      0.92      0.92    174304

[[57146  6042     6   320]
 [ 5333 78742   560   410]
 [    8   612 16138    28]
 [  561   656    80  7662]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.423873
1,Aspect,0.013340
2,Slope,0.000704
3,Horizontal_Distance_To_Hydrology,0.014165
4,Vertical_Scaled,0.023879
5,Horizontal_Distance_To_Roadways,0.083519
6,Hillshade_9am,0.022137
7,Hillshade_Noon,0.017396
8,Hillshade_3pm,0.012002
9,Horizontal_Distance_To_Fire_Points,0.054779


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9639618596142687
Testing Score: 0.9444361575178998


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.92      0.94     63514
           2       0.93      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.96      0.86      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[58534  4830     9   141]
 [ 2576 81940   377   152]
 [    0   324 16457     5]
 [  456   735    80  7688]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.279158
1,Aspect,0.020532
2,Slope,0.001374
3,Horizontal_Distance_To_Hydrology,0.022255
4,Vertical_Scaled,0.025170
5,Horizontal_Distance_To_Roadways,0.066961
6,Hillshade_9am,0.019895
7,Hillshade_Noon,0.016980
8,Hillshade_3pm,0.017283
9,Horizontal_Distance_To_Fire_Points,0.050663


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_b.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_b['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_b[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Slope,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Fire_Hydr,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,0.164973,-0.867052,-1.395274,0.330743,0.439143,0.142960,2.587999,1.300680,1.054833,1.068195,0.530152,0.135858,2.574818,29,1
1,-1.319235,-0.890480,-0.578038,-0.056053,-1.014305,-1.567204,0.293388,0.590899,0.221342,2.564187,-0.148505,0.979149,1.008612,0.645367,-0.082435,2.529746,29,1
2,-0.554907,-0.148836,-1.277558,0.210340,0.464875,0.655217,0.816364,0.742654,-0.196691,2.518036,0.587323,1.849922,1.855709,0.668410,0.215057,2.507987,12,1
3,-0.622768,-0.005869,0.943243,0.090516,1.305345,0.606471,0.965786,0.742654,-0.536343,2.557997,0.819799,1.842302,1.855709,0.461022,0.186241,2.536981,30,1
4,-1.301377,-0.988770,-0.578038,-0.378421,-0.891238,-1.565670,0.293388,0.540313,0.195215,2.540716,1.205062,0.940499,0.990654,0.599281,-0.401841,2.478848,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9298180190930787


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.93      0.92      0.92     63514
           2       0.93      0.94      0.93     85045
           3       0.96      0.97      0.96     16786
           4       0.90      0.87      0.89      8959

    accuracy                           0.93    174304
   macro avg       0.93      0.92      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[58187  4946    21   360]
 [ 4144 79845   636   420]
 [   25   505 16202    54]
 [  474   582    66  7837]]


### Ensemble techniques 

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.945348997315027
Training Score: 0.9685450003442273
Testing Score: 0.9484349183036533


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.95      0.88      0.91      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.94      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59231  4092     7   184]
 [ 2722 81753   361   209]
 [    4   304 16472     6]
 [  420   608    71  7860]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.933254324970249
Training Score: 0.9503402932816665
Accuracy: 0.9336331925830732


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.81      0.88      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.91      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[57703  5636    10   165]
 [ 3041 81375   489   140]
 [    1   414 16369     2]
 [  604   989    77  7289]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9289539423861837
Training Score: 0.9998475564778662
Testing Score: 0.930500734349183


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.93      0.91      0.92     63514
           2       0.93      0.94      0.94     85045
           3       0.96      0.97      0.96     16786
           4       0.91      0.87      0.89      8959

    accuracy                           0.93    174304
   macro avg       0.93      0.92      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[58107  5037    21   349]
 [ 3946 80046   652   401]
 [   27   499 16212    48]
 [  470   594    70  7825]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 0.9999975412335139
AdaBoost DT Testing Score: 0.9449180741692675


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.97     16786
           4       0.98      0.85      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.96      0.93      0.94    174304
weighted avg       0.95      0.94      0.94    174304

[[59158  4265     1    90]
 [ 3005 81512   445    83]
 [    3   375 16402     6]
 [  484   785    59  7631]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9516336044533179
AdaBoost RF Testing Score: 0.9126870295575547


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.91      0.88      0.89     63514
           2       0.90      0.94      0.92     85045
           3       0.97      0.98      0.98     16786
           4       0.98      0.77      0.86      8959

    accuracy                           0.91    174304
   macro avg       0.94      0.89      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[55789  7653     3    69]
 [ 4548 79967   450    80]
 [    1   308 16469     8]
 [ 1243   808    48  6860]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9756115751789977


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.93      0.95      8959

    accuracy                           0.98    174304
   macro avg       0.98      0.97      0.97    174304
weighted avg       0.98      0.98      0.98    174304

[[61658  1739     1   116]
 [ 1217 83475   210   143]
 [    0   188 16592     6]
 [  227   361    43  8328]]
              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.93      0.95      8959

    accuracy                           0.98    174304
   macro avg       0.98      0.97      0.97    174304
weighted avg       0.98      0.98      0.98    174304

[[61658  1739     1  

## Original Transformed features

Constructing a dataframe with original features after transformation

In [ ]:
forest_d = forest_b.drop(columns = forest_b.columns[10:16])
forest_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    581012 non-null  float64 
 4   Vertical_Scaled                     581012 non-null  float64 
 5   Horizontal_Distance_To_Roadways     581012 non-null  float64 
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  float64 
 10  Wilderness                          581012 non-null  category
 11  Soil         

## Modelling on Original Transformed Features

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_d.drop(columns = ['Cover_Type'])
y = forest_d['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9351721628293518
Testing Score: 0.9111035891316321


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.89      0.90     63514
           2       0.91      0.92      0.92     85045
           3       0.97      0.96      0.96     16786
           4       0.90      0.86      0.88      8959

    accuracy                           0.91    174304
   macro avg       0.92      0.91      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56755  6341    19   399]
 [ 5899 78253   484   409]
 [   14   626 16121    25]
 [  521   696    62  7680]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.431695
1,Aspect,0.017499
2,Slope,0.000859
3,Horizontal_Distance_To_Hydrology,0.047369
4,Vertical_Scaled,0.030493
5,Horizontal_Distance_To_Roadways,0.124322
6,Hillshade_9am,0.023697
7,Hillshade_Noon,0.025402
8,Hillshade_3pm,0.015113
9,Horizontal_Distance_To_Fire_Points,0.120681


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9523909045310148
Testing Score: 0.931992381127226


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.90      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.82      0.88      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.91      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[57387  5944    18   165]
 [ 3107 81334   457   147]
 [    1   410 16370     5]
 [  572   962    66  7359]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.302668
1,Aspect,0.028356
2,Slope,0.003151
3,Horizontal_Distance_To_Hydrology,0.042356
4,Vertical_Scaled,0.036576
5,Horizontal_Distance_To_Roadways,0.107635
6,Hillshade_9am,0.025252
7,Hillshade_Noon,0.027604
8,Hillshade_3pm,0.023500
9,Horizontal_Distance_To_Fire_Points,0.099782


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_d.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_d['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_d[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,0.164973,-0.867052,-1.395274,0.330743,0.439143,0.142960,2.587999,29,1
1,-1.319235,-0.890480,-0.578038,-0.056053,-1.014305,-1.567204,0.293388,0.590899,0.221342,2.564187,29,1
2,-0.554907,-0.148836,-1.277558,0.210340,0.464875,0.655217,0.816364,0.742654,-0.196691,2.518036,12,1
3,-0.622768,-0.005869,0.943243,0.090516,1.305345,0.606471,0.965786,0.742654,-0.536343,2.557997,30,1
4,-1.301377,-0.988770,-0.578038,-0.378421,-0.891238,-1.565670,0.293388,0.540313,0.195215,2.540716,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9260544795300165


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.92      0.91      0.92     63514
           2       0.93      0.94      0.93     85045
           3       0.96      0.96      0.96     16786
           4       0.90      0.86      0.88      8959

    accuracy                           0.93    174304
   macro avg       0.93      0.92      0.92    174304
weighted avg       0.93      0.93      0.93    174304

[[57831  5264    29   390]
 [ 4327 79666   632   420]
 [   23   530 16192    41]
 [  525   644    64  7726]]


### Ensemble Techniques

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9421255544518427
Training Score: 0.965574810429104
Testing Score: 0.9447115384615384


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.95      0.87      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[58881  4418     6   209]
 [ 2888 81617   350   190]
 [   11   359 16407     9]
 [  453   671    73  7762]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.918735308870246
Training Score: 0.9361482931243054
Accuracy: 0.9193822287497705


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     63514
           2       0.90      0.95      0.93     85045
           3       0.96      0.97      0.96     16786
           4       0.96      0.77      0.86      8959

    accuracy                           0.92    174304
   macro avg       0.94      0.89      0.91    174304
weighted avg       0.92      0.92      0.92    174304

[[56476  6827    16   195]
 [ 3740 80591   587   127]
 [    1   537 16245     3]
 [  734  1213    72  6940]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9247224052637273
Training Score: 0.9997811697827433
Testing Score: 0.9263183862676703


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.93      0.91      0.92     63514
           2       0.92      0.94      0.93     85045
           3       0.96      0.97      0.96     16786
           4       0.90      0.86      0.88      8959

    accuracy                           0.93    174304
   macro avg       0.93      0.92      0.92    174304
weighted avg       0.93      0.93      0.93    174304

[[57679  5429    29   377]
 [ 4105 79886   646   408]
 [   22   518 16206    40]
 [  543   657    69  7690]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 0.9999975412335139
AdaBoost DT Testing Score: 0.9502535799522673


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.98      0.98      0.98     16786
           4       0.97      0.87      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.96      0.94      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59495  3908     3   108]
 [ 2683 81923   346    93]
 [    3   324 16451     8]
 [  438   697    60  7764]]


#### Random Forest


In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9222341335798656
AdaBoost RF Testing Score: 0.8827565632458234


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.87      0.84      0.85     63514
           2       0.87      0.92      0.89     85045
           3       0.96      0.97      0.97     16786
           4       0.97      0.68      0.80      8959

    accuracy                           0.88    174304
   macro avg       0.92      0.85      0.88    174304
weighted avg       0.88      0.88      0.88    174304

[[53436  9972     6   100]
 [ 6390 77945   615    95]
 [    1   416 16356    13]
 [ 1739  1035    54  6131]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9729782449054526


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.98     16786
           4       0.97      0.92      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.97    174304
weighted avg       0.97      0.97      0.97    174304

[[61472  1911     3   128]
 [ 1372 83309   222   142]
 [    4   221 16548    13]
 [  258   383    53  8265]]


## Added Features

Constructing a dataframe with added features and removing features on which some feature engineering has been applied, so as to gauge their performance in classifying cover_types.

In [ ]:
forest_c = forest_b.drop(columns = forest_b.columns[3:10])
forest_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Elevation       581012 non-null  int64   
 1   Aspect          581012 non-null  int64   
 2   Slope           581012 non-null  float64 
 3   Hydr_Slope      581012 non-null  float64 
 4   Avg_Horizontal  581012 non-null  float64 
 5   Fire_Road       581012 non-null  float64 
 6   Hillshades      581012 non-null  float64 
 7   Hydr_Dist       581012 non-null  float64 
 8   Fire_Hydr       581012 non-null  float64 
 9   Wilderness      581012 non-null  category
 10  Soil            581012 non-null  category
 11  Cover_Type      581012 non-null  category
dtypes: category(3), float64(7), int64(2)
memory usage: 41.6 MB


## Modelling on added features

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_c.drop(columns = ['Cover_Type'])
y = forest_c['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9310168474679623
Testing Score: 0.9055271250229484


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.89      0.89      0.89     63514
           2       0.91      0.91      0.91     85045
           3       0.95      0.96      0.95     16786
           4       0.89      0.85      0.87      8959

    accuracy                           0.91    174304
   macro avg       0.91      0.90      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56639  6448     8   419]
 [ 6254 77522   776   493]
 [   19   653 16084    30]
 [  535   756    76  7592]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.438735
1,Aspect,0.043515
2,Slope,0.001225
3,Hydr_Slope,0.003349
4,Avg_Horizontal,0.063481
5,Fire_Road,0.093511
6,Hillshades,0.029338
7,Hydr_Dist,0.052525
8,Fire_Hydr,0.103629
9,Wilderness,0.045393


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9546210057338435
Testing Score: 0.9331856985496604


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63514
           2       0.92      0.95      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.84      0.89      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.92      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[57649  5684     6   175]
 [ 3299 81116   476   154]
 [    1   389 16392     4]
 [  502   880    76  7501]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.312647
1,Aspect,0.043193
2,Slope,0.003384
3,Hydr_Slope,0.007781
4,Avg_Horizontal,0.084352
5,Fire_Road,0.094981
6,Hillshades,0.032670
7,Hydr_Dist,0.050086
8,Fire_Hydr,0.091183
9,Wilderness,0.081561


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_c.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_c['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_c[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Hydr_Slope,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Fire_Hydr,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,1.300680,1.054833,1.068195,0.530152,0.135858,2.574818,29,1
1,-1.319235,-0.890480,-0.578038,-0.148505,0.979149,1.008612,0.645367,-0.082435,2.529746,29,1
2,-0.554907,-0.148836,-1.277558,0.587323,1.849922,1.855709,0.668410,0.215057,2.507987,12,1
3,-0.622768,-0.005869,0.943243,0.819799,1.842302,1.855709,0.461022,0.186241,2.536981,30,1
4,-1.301377,-0.988770,-0.578038,1.205062,0.940499,0.990654,0.599281,-0.401841,2.478848,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9120502111253901


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90     63514
           2       0.91      0.92      0.92     85045
           3       0.95      0.96      0.95     16786
           4       0.87      0.84      0.86      8959

    accuracy                           0.91    174304
   macro avg       0.91      0.90      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56782  6196    29   507]
 [ 5164 78581   765   535]
 [   23   652 16069    42]
 [  569   779    69  7542]]


### Ensemble techniques

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9388332661270494
Training Score: 0.9635856683419063
Testing Score: 0.9418544611712869


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.93      0.93     63514
           2       0.94      0.95      0.95     85045
           3       0.97      0.98      0.97     16786
           4       0.95      0.87      0.91      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304

[[58778  4520     6   210]
 [ 3256 81179   409   201]
 [    9   357 16413     7]
 [  420   659    81  7799]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9208228016168848
Training Score: 0.9385849307119605
Accuracy: 0.9211549935744446


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     63514
           2       0.91      0.95      0.93     85045
           3       0.96      0.97      0.96     16786
           4       0.95      0.79      0.86      8959

    accuracy                           0.92    174304
   macro avg       0.94      0.90      0.92    174304
weighted avg       0.92      0.92      0.92    174304

[[56691  6605     7   211]
 [ 3782 80484   631   148]
 [    1   494 16288     3]
 [  660  1125    76  7098]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9111254265959853
Training Score: 0.9997713347167992
Testing Score: 0.9126526069395998


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90     63514
           2       0.91      0.93      0.92     85045
           3       0.95      0.96      0.95     16786
           4       0.88      0.84      0.86      8959

    accuracy                           0.91    174304
   macro avg       0.91      0.90      0.91    174304
weighted avg       0.91      0.91      0.91    174304

[[56650  6376    26   462]
 [ 4946 78847   776   476]
 [   21   638 16087    40]
 [  593   797    74  7495]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 0.9998475564778662
AdaBoost DT Testing Score: 0.9328070497521571


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.91      0.92     63514
           2       0.92      0.95      0.94     85045
           3       0.97      0.97      0.97     16786
           4       0.97      0.83      0.90      8959

    accuracy                           0.93    174304
   macro avg       0.95      0.92      0.93    174304
weighted avg       0.93      0.93      0.93    174304

[[58113  5291     2   108]
 [ 3746 80708   502    89]
 [    0   448 16330     8]
 [  595   864    59  7441]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9239306824552259
AdaBoost RF Testing Score: 0.8834507527079126


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.88      0.83      0.85     63514
           2       0.86      0.93      0.89     85045
           3       0.96      0.98      0.97     16786
           4       0.97      0.71      0.82      8959

    accuracy                           0.88    174304
   macro avg       0.92      0.86      0.88    174304
weighted avg       0.89      0.88      0.88    174304

[[52494 10921     1    98]
 [ 5520 78777   651    97]
 [    1   378 16399     8]
 [ 1558  1029    53  6319]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9706145584725537


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.96      0.97     63514
           2       0.97      0.98      0.97     85045
           3       0.98      0.99      0.99     16786
           4       0.96      0.92      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.97    174304
weighted avg       0.97      0.97      0.97    174304

[[61215  2163     3   133]
 [ 1515 83116   239   175]
 [    5   200 16573     8]
 [  253   380    48  8278]]


## Feature Selection

Selecting features on basis of their importance in decision tree and random forest models. Also, considering the overall performance of the models.

In [ ]:
forest_e = forest_b.drop(columns = forest_b.columns[2:5])
forest_e = forest_e.drop(columns = forest_b.columns[7:9])
forest_e = forest_e.drop(columns = forest_b.columns[10])
forest_e = forest_e.drop(columns = forest_b.columns[13])
forest_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 12 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Horizontal_Distance_To_Roadways     581012 non-null  float64 
 3   Hillshade_9am                       581012 non-null  int64   
 4   Horizontal_Distance_To_Fire_Points  581012 non-null  float64 
 5   Avg_Horizontal                      581012 non-null  float64 
 6   Fire_Road                           581012 non-null  float64 
 7   Hydr_Dist                           581012 non-null  float64 
 8   Fire_Hydr                           581012 non-null  float64 
 9   Wilderness                          581012 non-null  category
 10  Soil                                581012 non-null  category
 11  Cover_Type   

## Modelling after Feature Selection

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_e.drop(columns = ['Cover_Type'])
y = forest_e['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9382579147693185
Testing Score: 0.9158022764824674


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.92      0.92      0.92     85045
           3       0.96      0.96      0.96     16786
           4       0.90      0.87      0.89      8959

    accuracy                           0.92    174304
   macro avg       0.92      0.91      0.92    174304
weighted avg       0.92      0.92      0.92    174304

[[57388  5755    18   353]
 [ 5804 78261   534   446]
 [   14   579 16165    28]
 [  430   645    70  7814]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.427199
1,Aspect,0.033876
2,Horizontal_Distance_To_Roadways,0.085645
3,Hillshade_9am,0.026552
4,Horizontal_Distance_To_Fire_Points,0.062435
5,Avg_Horizontal,0.040512
6,Fire_Road,0.071148
7,Hydr_Dist,0.048804
8,Fire_Hydr,0.043741
9,Wilderness,0.042433


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9652920522832106
Testing Score: 0.9477235175325868


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.96      0.87      0.91      8959

    accuracy                           0.95    174304
   macro avg       0.96      0.93      0.94    174304
weighted avg       0.95      0.95      0.95    174304

[[58978  4359     4   173]
 [ 2509 82005   368   163]
 [    0   331 16450     5]
 [  404   719    77  7759]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.293586
1,Aspect,0.035508
2,Horizontal_Distance_To_Roadways,0.074809
3,Hillshade_9am,0.028406
4,Horizontal_Distance_To_Fire_Points,0.055968
5,Avg_Horizontal,0.074931
6,Fire_Road,0.072505
7,Hydr_Dist,0.043682
8,Fire_Hydr,0.055738
9,Wilderness,0.075564


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_e.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_e['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_e[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Horizontal_Distance_To_Roadways,Hillshade_9am,Horizontal_Distance_To_Fire_Points,Avg_Horizontal,Fire_Road,Hydr_Dist,Fire_Hydr,Soil,Wilderness
0,-1.297805,-0.935157,-1.395274,0.330743,2.587999,1.054833,1.068195,0.135858,2.574818,29,1
1,-1.319235,-0.890480,-1.567204,0.293388,2.564187,0.979149,1.008612,-0.082435,2.529746,29,1
2,-0.554907,-0.148836,0.655217,0.816364,2.518036,1.849922,1.855709,0.215057,2.507987,12,1
3,-0.622768,-0.005869,0.606471,0.965786,2.557997,1.842302,1.855709,0.186241,2.536981,30,1
4,-1.301377,-0.988770,-1.565670,0.293388,2.540716,0.940499,0.990654,-0.401841,2.478848,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9659043969157334


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.96      0.96      0.96     63514
           2       0.97      0.97      0.97     85045
           3       0.98      0.98      0.98     16786
           4       0.94      0.93      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.96      0.96      0.96    174304
weighted avg       0.97      0.97      0.97    174304

[[60955  2346    10   203]
 [ 2004 82550   230   261]
 [    8   236 16500    42]
 [  241   322    40  8356]]


### Ensemble Techniques

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9460743334284056
Training Score: 0.9678590044946251
Testing Score: 0.9489684688819534


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.94     63514
           2       0.95      0.96      0.95     85045
           3       0.97      0.98      0.98     16786
           4       0.95      0.88      0.92      8959

    accuracy                           0.95    174304
   macro avg       0.95      0.94      0.95    174304
weighted avg       0.95      0.95      0.95    174304

[[59508  3809     4   193]
 [ 2935 81524   368   218]
 [    5   311 16463     7]
 [  374   598    73  7914]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9389832508826972
Training Score: 0.9532662254000414
Accuracy: 0.9389973838810355


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.92      0.93     63514
           2       0.93      0.96      0.94     85045
           3       0.97      0.98      0.97     16786
           4       0.96      0.83      0.89      8959

    accuracy                           0.94    174304
   macro avg       0.95      0.92      0.93    174304
weighted avg       0.94      0.94      0.94    174304

[[58276  5039     9   190]
 [ 2895 81548   448   154]
 [    0   405 16379     2]
 [  524   885    82  7468]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9640454576747937
Training Score: 0.9999114844065029
Testing Score: 0.9658011290618689


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.96      0.96      0.96     63514
           2       0.97      0.97      0.97     85045
           3       0.98      0.98      0.98     16786
           4       0.94      0.93      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.96      0.96      0.96    174304
weighted avg       0.97      0.97      0.97    174304

[[60908  2396    11   199]
 [ 1960 82593   235   257]
 [    8   244 16498    36]
 [  246   322    47  8344]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 1.0
AdaBoost DT Testing Score: 0.9670346062052506


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.96      0.96     63514
           2       0.96      0.98      0.97     85045
           3       0.98      0.99      0.98     16786
           4       0.97      0.91      0.94      8959

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.96    174304
weighted avg       0.97      0.97      0.97    174304

[[60862  2551     0   101]
 [ 1733 82953   250   109]
 [    1   226 16551     8]
 [  258   463    46  8192]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.9280146935885205
AdaBoost RF Testing Score: 0.9030372223242151


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.90      0.86      0.88     63514
           2       0.89      0.94      0.91     85045
           3       0.97      0.98      0.98     16786
           4       0.97      0.75      0.85      8959

    accuracy                           0.90    174304
   macro avg       0.93      0.88      0.90    174304
weighted avg       0.90      0.90      0.90    174304

[[54617  8832     0    65]
 [ 4874 79600   447   124]
 [    0   293 16489     4]
 [ 1449   766    47  6697]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9753419313383513


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.98      0.99      0.99     16786
           4       0.97      0.93      0.95      8959

    accuracy                           0.98    174304
   macro avg       0.98      0.97      0.97    174304
weighted avg       0.98      0.98      0.98    174304

[[61640  1742     6   126]
 [ 1271 83404   214   156]
 [    0   187 16589    10]
 [  196   353    37  8373]]
